In [ ]:
import sys, pathlib
if str(pathlib.Path().resolve().parent) not in sys.path:
    sys.path.append(str(pathlib.Path().resolve().parent))

In [ ]:
import time
import pandas as pd
import plotly.graph_objects as go

from exchange_api.futures import FuturesClient

In [ ]:
futures_client = FuturesClient()

In [ ]:
futures_client.ping()

In [ ]:
pd.DataFrame(
    futures_client.contracts()
)

In [ ]:
pd.DataFrame(
    futures_client.ticker_all()
).T

In [ ]:
futures_client.index_price('E-ETH-USDT')

In [ ]:
futures_client.klines('E-ETH-USDT', '30min', 2)

In [ ]:
dt = 60 * 60 * 1000
data = futures_client.klines('E-ETH-USDT', '30min', 100)
data = sorted(data, key=lambda x: x['idx'])

df = pd.DataFrame(data)
df = df.astype({'open': float, 'high': float, 'low': float, 'close': float, 'vol': float})
df['time'] = pd.to_datetime(df['idx'], unit='ms', utc=True).dt.tz_convert('Asia/Taipei')
df['MA20'] = df['close'].rolling(window=20).mean()
df = df.sort_values('time')
fig = go.Figure(data=[
    go.Candlestick(x=df['time'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], name='K-Line')
])
fig.add_trace(go.Bar(x=df['time'], y=df['vol'], name='Volume', yaxis='y2', opacity=0.2))
fig.add_trace(go.Scatter(x=df['time'], y=df['MA20'], mode='lines', name='MA20', yaxis='y1', line=dict(color='#F0F')))
fig.update_layout(
    width=1080, height=500, yaxis_title='Price', xaxis_rangeslider_visible=False, 
    yaxis2=dict(title='Volume', overlaying='y', side='right', showgrid=False),
    xaxis=dict(tickformat='%m/%d-%H:%M', tickangle=-45, tickfont=dict(size=10), dtick=dt, showgrid=True,),
)
fig.show()

# 交易相關

詳情請見 [API 文檔](https://exchangedocsv2.gitbook.io/open-api-doc-v2/jian-ti-zhong-wen-v2/he-yue-jiao-yi#jiao-yi-xiang-guan)


<div style="font-size:12pt; color: red; font-weight: bold">
    <div style=" border: 1px solid red; border-radius: 10px; padding: 20px; width: 30rem; margin: auto; text-align: center">
    注意！執行以下命令行會直接下單，在 App 當中可以即時看到，<br>這裡使用模擬幣對 S-BTC-USDT 做範例。
    </div>
</div>

In [ ]:
contract_name = 'S-BTC-USDT'

## 更改持倉模式

In [ ]:
futures_client.edit_user_position_model(contract_name, 1)

## A. 做多開倉

In [ ]:
futures_client.create_order(
    volume=1,
    order_unit=2,    # 張（合約）
    position_type=1, # 逐倉
    leverage=100,    # 槓桿
    side="BUY",      # 多
    open="OPEN",     # 開倉
    type="MARKET",   # 市價單
    contract_name=contract_name
)

## A. 做空平倉

對上一格的下單做平倉

In [ ]:
futures_client.create_order(
    volume=1,
    order_unit=2,    # 張（合約）
    position_type=1, # 逐倉
    side="SELL",     # 反方向
    open="CLOSE",    # 平倉
    type="MARKET",   # 市價單
    contract_name=contract_name
)

## B. 做空開倉

In [ ]:
futures_client.create_order(
    volume=1,
    order_unit=2,    # 張（合約）
    position_type=1, # 逐倉
    leverage=100,    # 槓桿
    side="SELL",     # 空
    open="OPEN",     # 開倉
    type="MARKET",   # 市價單
    contract_name=contract_name
)

## B. 做多反向平倉

對上一格的下單做反向單，這一格跑完記得可以去 App 手動平倉

In [ ]:
futures_client.create_order(
    volume=2,
    order_unit=2,    # 張（合約）
    position_type=1, # 逐倉
    side="BUY",      # 多
    open="OPEN",     # 開倉
    type="MARKET",   # 市價單
    contract_name=contract_name
)

# 獲取目前的所有訂單

In [ ]:
pd.DataFrame(futures_client.account()['account'])